In [5]:
import configparser
import pymysql
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
def create_connection():
    config = configparser.ConfigParser()
    config.read('../Config_db/config.ini')
    host = config['mysql']['host']
    user = config['mysql']['user']
    password = config['mysql']['password']
    database = config['mysql']['database']
    try:
        conn = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        print("Successful Connection")
        cursor = conn.cursor()
        return conn, cursor
    except pymysql.Error as e:
        print("Connection Error:", e)
        return None, None
    
def query_db(dbname):
    conn, cursor = create_connection()
    cursor.execute(dbname)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    conn.close()
    return df

In [11]:
dbname = ('SELECT * FROM grammy')
grammy = query_db(dbname)

Successful Connection


In [12]:
grammy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   year          4810 non-null   int64         
 1   title         4810 non-null   object        
 2   published_at  4810 non-null   datetime64[ns]
 3   updated_at    4810 non-null   datetime64[ns]
 4   category      4810 non-null   object        
 5   nominee       4810 non-null   object        
 6   artist        4810 non-null   object        
 7   workers       4810 non-null   object        
 8   img           4810 non-null   object        
 9   winner        4810 non-null   int64         
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 375.9+ KB
